# Telegram相关

In [ ]:
# 将用于telegram查询群组的关键词从csv提取到json
import csv

results={}
results["all"]=[]
regex_value=""
with open("dark_keywords_telegram_search/dark_keywords.csv",) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        results.setdefault(row["category"],[])
        results[row["category"]].append(row["keyword"])
        results["all"].append(row["keyword"])
        regex_value+=(row["keyword"]+"|")
        # print(regex_value)
csvfile.close()
print(regex_value[:-1])
results["regex"]=regex_value[:-1]

results

import json

with open("dark_keywords_telegram_search/dark_keywords.json","w") as jsonfile:
    json.dump(results,jsonfile,ensure_ascii=False)
jsonfile.close()

In [ ]:
# 将查询出来的group_channle_url汇总成regex表达式
import csv
results={}
regex_value=""
with open("dark_keywords_telegram_search/keyword_group_channel_url.csv",) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        regex_value+=(row["group_channel_url"]+"|")
        # print(regex_value)
csvfile.close()
print(regex_value[:-1])
results["regex"]=regex_value[:-1]

import json

with open("dark_keywords_telegram_search/keyword_group_channel_url.json","w") as jsonfile:
    json.dump(results,jsonfile,ensure_ascii=False)
jsonfile.close()

In [ ]:
# 将group_channel

# Atom相关

In [ ]:
# 将整理的用于大模型微调的黑话origin数据集整理成一列：csv->csv
import csv

results=[["text"]]
with open("darkword_data_collected/darkword_origin_data.csv","r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text="<s>Human: "+row["background"]+row["category"]+"，"+row["input"]+"\n</s><s>Assistant: "+row["content"]+"\n</s>"
        print(text)
        results.append([text])
csvfile.close()

with open("darkword_data_collected/darkword_train_data.csv","w") as file:
    writer = csv.writer(file)
    writer.writerows(results)

In [ ]:
# 将整理的用于大模型微调的黑话origin数据集整理成一列：csv->csv（version3）
# <s>{category}的{input}</s><s>{content}</s>
import csv

results=[["text"]]
with open("darkword_origin_data.csv","r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text="<s>Human: "+row["category"]+"的"+row["input"]+"\n</s><s>Assistant: "+row["content"]+"\n</s>"
        print(text)
        results.append([text])
csvfile.close()

with open("darkword_data_collected/darkword_format_data.csv","w") as file:
    writer = csv.writer(file)
    writer.writerows(results)

In [ ]:
# atom-7b
# 将格式化后好的数据按比例划分为训练数据集合验证数据集合。
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("darkword_data_atom/darkword_format_data.csv")
train_df, valid_df = train_test_split(df,test_size=0.1, random_state=51)
train_df.to_csv("darkword_data_atom/darkword_train_data.csv",index=False)
valid_df.to_csv("darkword_data_atom/darkword_validate_data.csv",index=False)

# Chatglm3相关

In [3]:
# 第一版测试，将所有conversation都存入一个数组[]，再写入json文件
# chatglm3-6b数据集处理
# 格式：{"conversations":[{"role":"user","content":row["content"]},{"role":"assistant","content":row["summary"]},{},{}]}
import csv,json

# origin_data_file="darkword_origin_data.csv"
origin_data_file="test_origin_data.csv"
output_data_dir="darkword_data_chatglm3/"
output_data_file=output_data_dir+"format.json"
train_data_file=output_data_dir+"train.json"
validation_data_file=output_data_dir+"dev.json"
results=[]
with open(origin_data_file,"r",encoding="utf-8") as csvfile:
    # with open(output_data_file,"wt",encoding="utf-8") as outputfile:
    #     reader=csv.DictReader(csvfile)
    #     for row in reader:
    #         sample={"conversations":[{"role":"user","content":row["category"]+"的"+row["input"]},{"role":"assistant","content":row["content"]}]}
    #         outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
    # outputfile.close()
    reader = csv.DictReader(csvfile)
    for row in reader:
        sample={"conversations":[{"role":"user","content":row["category"]+"的"+row["input"]},{"role":"assistant","content":row["content"]}]}
        results.append(sample)
csvfile.close()
with open(output_data_file,"wt",encoding="utf-8") as output:
    json.dump(results,output,ensure_ascii=False,indent=4)
output.close()

# atom-7b
# 将格式化后好的数据按比例随机划分为训练数据集合验证数据集合。
import random

data=[]
with open(output_data_file,"r",encoding="utf-8") as readfile:
    data=json.load(readfile)
readfile.close()
# data
# 打乱
random.shuffle(data)
# 定义比例
train_ratio = 0.9
train_size = int(len(data)*train_ratio)
# 分割
train_data=data[:train_size]
validation_data=data[train_size:]

with open(train_data_file,"wt",encoding="utf-8") as outputfile:
    # for sample in train_data:
    #     outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
    json.dump(train_data,outputfile,ensure_ascii=False,indent=4)
outputfile.close()
with open(validation_data_file,"wt",encoding="utf-8") as outputfile:
    # for sample in validation_data:
    #     outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
    json.dump(validation_data,outputfile,ensure_ascii=False,indent=4)
outputfile.close()

In [12]:
# 第二版测试，将每个conversation一行一行的存入json文件中。
# chatglm3-6b数据集处理
# 格式：{"conversations":[{"role":"user","content":row["content"]},{"role":"assistant","content":row["summary"]},{},{}]}
import csv,json

origin_data_file="darkword_origin_data.csv"
# origin_data_file="test_origin_data.csv"
output_data_dir="darkword_data_chatglm3/"
output_data_file=output_data_dir+"format.json"
train_data_file=output_data_dir+"train.json"
validation_data_file=output_data_dir+"dev.json"
results=[]
with open(origin_data_file,"r",encoding="utf-8") as csvfile:
    with open(output_data_file,"wt",encoding="utf-8") as outputfile:
        reader=csv.DictReader(csvfile)
        for row in reader:
            sample={"conversations":[{"role":"user","content":row["category"]+"的"+row["input"]},{"role":"assistant","content":row["content"]}]}
            outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
    outputfile.close()
csvfile.close()

# atom-7b
# 将格式化后好的数据按比例随机划分为训练数据集合验证数据集合。
import random

data=[]
with open(output_data_file,"r",encoding="utf-8") as readfile:
    data=[json.loads(line) for line in readfile]
readfile.close()
# data
# 打乱
random.shuffle(data)
# 定义比例
train_ratio = 0.9
train_size = int(len(data)*train_ratio)
# 分割
train_data=data[:train_size]
validation_data=data[train_size:]

with open(train_data_file,"wt",encoding="utf-8") as outputfile:
    for sample in train_data:
        outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
outputfile.close()
with open(validation_data_file,"wt",encoding="utf-8") as outputfile:
    for sample in validation_data:
        outputfile.write(json.dumps(sample,ensure_ascii=False)+"\n")
outputfile.close()

# Baichuan2相关

In [4]:
# 将原始数据集转换为baichun2微调的格式
# 格式：[{"id":*,"conversations":[{"from":"*","value":"*"},{"from":"gpt","value":"*"},{},{}]},{},{}]
import csv,json

results=[]
with open("darkword_origin_data.csv","r") as csvfile:
    reader = csv.DictReader(csvfile)
    for index,row in enumerate(reader,start=1):
        conversation=[{"from":"human","value":row["category"]+"的"+row["input"]+"\n"},{"from":"gpt","value":row["content"]+"\n"}]
        one_result={"id":index,"conversations":conversation}
        results.append(one_result)
csvfile.close()

with open("darkword_data_baichuan2/train_data.json","w",encoding="utf-8") as output:
    json.dump(results,output,ensure_ascii=False,indent=4)
output.close()